## Slice the large CSV into a data cube with figures aggregated

In [2]:
# import libraries
import pandas as pd

# read data
df = pd.read_csv('../../../data/crashes.csv')

## Loading checking

In [32]:
print(df.head())
df.shape

                                     CRASH_RECORD_ID     RD_NO  \
0  79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...  JC199149   
1  792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...  JB422857   
2  0115ade9a755e835255508463f7e9c4a9a0b47e9304238...  JF318029   
3  017040c61958d2fa977c956b2bd2d6759ef7754496dc96...  JF324552   
4  78eee027ec3dcc85d36c9e3fdae4729dcc56440105d65b...  JB291672   

  CRASH_DATE_EST_I  POSTED_SPEED_LIMIT TRAFFIC_CONTROL_DEVICE  \
0              NaN                  30         TRAFFIC SIGNAL   
1              NaN                  30            NO CONTROLS   
2              NaN                  30                UNKNOWN   
3              NaN                  30         TRAFFIC SIGNAL   
4              NaN                  30            NO CONTROLS   

       DEVICE_CONDITION WEATHER_CONDITION      LIGHTING_CONDITION  \
0  FUNCTIONING PROPERLY             CLEAR                DAYLIGHT   
1           NO CONTROLS             CLEAR                DAYLIGHT   
2    

(541604, 53)

## People involved and injured

In [4]:
# calculate the total of row count, INJURIES_TOTAL, INJURIES_FATAL, INJURIES_INCAPACITATING, INJURIES_NON_INCAPACITATING, INJURIES_REPORTED_NOT_EVIDENT, INJURIES_NO_INDICATION, INJURIES_UNKNOWN
injuries = df.groupby(['CRASH_YEAR','SIDE']).agg({'INJURIES_TOTAL':'sum','INJURIES_FATAL':'sum','INJURIES_INCAPACITATING':'sum','INJURIES_NON_INCAPACITATING':'sum','INJURIES_REPORTED_NOT_EVIDENT':'sum','INJURIES_NO_INDICATION':'sum'}).reset_index()

# convert columns other than 'CRASH_YEAR', 'SIDE' to int
injuries.iloc[:,2:] = injuries.iloc[:,2:].fillna(0).astype(int)

# select columns to be used
crash_ppl_involved = injuries[['CRASH_YEAR','SIDE','INJURIES_TOTAL','INJURIES_NO_INDICATION']]
crash_ppl_injured = injuries[['CRASH_YEAR','SIDE','INJURIES_FATAL','INJURIES_INCAPACITATING','INJURIES_NON_INCAPACITATING','INJURIES_REPORTED_NOT_EVIDENT']]

# rename columns
crash_ppl_involved.columns = ['CRASH_YEAR','SIDE','Injured','No indication of injury']
crash_ppl_injured.columns = ['CRASH_YEAR','SIDE','Fatal','Incapacitating','Non-incapacitating','Reported but not evident']

# output to JSON
crash_ppl_involved.to_json('crash_ppl_involved.json',orient='records',indent=2)
crash_ppl_injured.to_json('crash_ppl_injured.json',orient='records',indent=2)

In [35]:

#divide the  INJURIES_FATAL, INJURIES_INCAPACITATING, INJURIES_NON_INCAPACITATING, INJURIES_REPORTED_NOT_EVIDENT fields by INJURIES_TOTAL and convert to percentage
injuries_percent = injuries.copy()
injuries_percent['FATAL'] = round((injuries_percent['INJURIES_FATAL']/injuries_percent['INJURIES_TOTAL'])*100,1)
injuries_percent['INCAPACITATING'] =round((injuries_percent['INJURIES_INCAPACITATING']/injuries_percent['INJURIES_TOTAL'])*100,1)
injuries_percent['NON_INCAPACITATING'] = round((injuries_percent['INJURIES_NON_INCAPACITATING']/injuries_percent['INJURIES_TOTAL'])*100,1)
injuries_percent['NOT_EVIDENT'] = round((injuries_percent['INJURIES_REPORTED_NOT_EVIDENT']/injuries_percent['INJURIES_TOTAL'])*100,1)
injuries_percent_clean = injuries_percent[['CRASH_YEAR','INJURIES_TOTAL','FATAL','INCAPACITATING','NON_INCAPACITATING','NOT_EVIDENT']]

print("Injuries proportion")
injuries_percent_clean.head(10)

Injuries proportion


,CRASH_YEAR,INJURIES_TOTAL,FATAL,INCAPACITATING,NON_INCAPACITATING,NOT_EVIDENT
0,2018,22424.0,0.6,11.1,56.1,32.3
1,2019,22270.0,0.5,10.6,56.4,32.5
2,2020,19622.0,0.7,11.3,58.1,29.8
3,2021,22205.0,0.7,10.6,58.9,29.8
4,2022,21858.0,0.7,10.3,57.7,31.2


## Time

In [5]:
#pivot CRASH_DAY_OF_WEEK and CRASH_HOUR to create a new column for each month and day
crash_day_time = df.pivot_table(index=['CRASH_YEAR','SIDE'], columns=['CRASH_DAY_OF_WEEK','CRASH_HOUR'], values='CRASH_RECORD_ID', aggfunc='count').reset_index()

#convert columns other than 'CRASH_YEAR', 'SIDE' to int
crash_day_time.iloc[:,2:] = crash_day_time.iloc[:,2:].fillna(0).astype(int)

#flatten the column to 1 layer and rename the columns of CRASH_DAY_OF_WEEK and CRASH_HOUR into a comma seperated string
crash_day_time.columns = ['CRASH_YEAR','SIDE', \
                          'Sun,0','Sun,1','Sun,2','Sun,3','Sun,4','Sun,5','Sun,6','Sun,7','Sun,8','Sun,9','Sun,10','Sun,11','Sun,12','Sun,13','Sun,14','Sun,15','Sun,16','Sun,17','Sun,18','Sun,19','Sun,20','Sun,21','Sun,22','Sun,23', \
                          'Mon,0','Mon,1','Mon,2','Mon,3','Mon,4','Mon,5','Mon,6','Mon,7','Mon,8','Mon,9','Mon,10','Mon,11','Mon,12','Mon,13','Mon,14','Mon,15','Mon,16','Mon,17','Mon,18','Mon,19','Mon,20','Mon,21','Mon,22','Mon,23', \
                          'Tue,0','Tue,1','Tue,2','Tue,3','Tue,4','Tue,5','Tue,6','Tue,7','Tue,8','Tue,9','Tue,10','Tue,11','Tue,12','Tue,13','Tue,14','Tue,15','Tue,16','Tue,17','Tue,18','Tue,19','Tue,20','Tue,21','Tue,22','Tue,23', \
                          'Wed,0','Wed,1','Wed,2','Wed,3','Wed,4','Wed,5','Wed,6','Wed,7','Wed,8','Wed,9','Wed,10','Wed,11','Wed,12','Wed,13','Wed,14','Wed,15','Wed,16','Wed,17','Wed,18','Wed,19','Wed,20','Wed,21','Wed,22','Wed,23', \
                          'Thu,0','Thu,1','Thu,2','Thu,3','Thu,4','Thu,5','Thu,6','Thu,7','Thu,8','Thu,9','Thu,10','Thu,11','Thu,12','Thu,13','Thu,14','Thu,15','Thu,16','Thu,17','Thu,18','Thu,19','Thu,20','Thu,21','Thu,22','Thu,23', \
                          'Fri,0','Fri,1','Fri,2','Fri,3','Fri,4','Fri,5','Fri,6','Fri,7','Fri,8','Fri,9','Fri,10','Fri,11','Fri,12','Fri,13','Fri,14','Fri,15','Fri,16','Fri,17','Fri,18','Fri,19','Fri,20','Fri,21','Fri,22','Fri,23', \
                          'Sat,0','Sat,1','Sat,2','Sat,3','Sat,4','Sat,5','Sat,6','Sat,7','Sat,8','Sat,9','Sat,10','Sat,11','Sat,12','Sat,13','Sat,14','Sat,15','Sat,16','Sat,17','Sat,18','Sat,19','Sat,20','Sat,21','Sat,22','Sat,23']
crash_day_time.head()

#output to JSON
crash_day_time.to_json('crash_day_time.json', orient='records',indent=2)

## Causes

In [30]:
causes = df.pivot_table(index=['CRASH_YEAR','SIDE'],columns='PRIM_CONTRIBUTORY_CAUSE',values='CRASH_RECORD_ID',aggfunc='count').reset_index()

# convert columns other than 'CRASH_YEAR', 'SIDE' to int
causes.iloc[:,2:] = causes.iloc[:,2:].fillna(0).astype(int)

# ensure the names are sorted in alphabetical order
cols = causes.columns.tolist()
# remove CRASH_YEAR and SIDE from the list
cols.remove('CRASH_YEAR')
cols.remove('SIDE')
# reorder the columns
cols.sort()
cols = ['CRASH_YEAR', 'SIDE'] + cols
causes = causes[cols]

# rename the columns
causes.columns = ['CRASH_YEAR','SIDE', \
                'Animal', \
                'Bicycle advancing legally on red light', \
                'Cell phone use other than texting', \
                'Disregarding other traffic signs', \
                'Disregarding road markings', \
                'Disregarding stop sign', \
                'Disregarding traffic signals', \
                'Disregarding yield sign', \
                'Distraction - from inside vehicle', \
                'Distraction - from outside vehicle', \
                'Distraction - other electronic device (navigation device, DVD player, etc.)', \
                'Driving on wrong side/wrong way', \
                'Driving skills/knowledge/experience', \
                'Equipment - vehicle condition', \
                'Evasive action due to animal, object, nonmotorist', \
                'Exceeding authorized speed limit', \
                'Exceeding safe speed for conditions', \
                'Failing to reduce speed to avoid crash', \
                'Failing to yield right-of-way', \
                'Following too closely', \
                'Had been drinking (not arrested)', \
                'Improper backing', \
                'Improper lane usage', \
                'Improper overtaking/passing', \
                'Improper turning/no signal', \
                'Motorcycle advancing legally on red light', \
                'Not applicable', \
                'Obstructed crosswalks', \
                'Operating vehicle in erratic, reckless, careless, negligent or aggressive manner', \
                'Passing stopped school bus', \
                'Physical condition of driver', \
                'Related to bus stop', \
                'Road construction/maintenance', \
                'Road engineering/surface/marking defects', \
                'Texting', \
                'Turning right on red', \
                'Unable to determine', \
                'Under the influence of alcohol/drugs (arrested)', \
                'Vision obscured (signs, tree limbs, buildings, etc.)', \
                'Weather']

# output to JSON
causes.to_json('crash_cause.json', orient='records',indent=2)